### The script will train a deep convolutional neural network to classifer each image to its architectural styles. It contrains two steps:
    1. Crate a classifer with transfer learning from Xception network. Freeze the base layer and only train the top layers.
    2. Unfreeze the base layer, use a very small learning rate to fine tuning the base layers.

In [ ]:
import os
from keras import preprocessing
import matplotlib.pyplot as plt
import shutil
import numpy as np
import keras
import pandas as pd
import cv2
from keras.losses import categorical_crossentropy
from keras.metrics import categorical_accuracy
from keras.callbacks import ModelCheckpoint
from webapp.utils import *

In [ ]:
# Create train and test generator
traingen = preprocessing.image.ImageDataGenerator(rescale=1/255.0,horizontal_flip=True,preprocessing_function=sketchify) #
testgen = preprocessing.image.ImageDataGenerator(rescale=1/255.0,horizontal_flip=True,preprocessing_function=sketchify)
train = '/scratch/zhang.chi9/train/'
test = '/scratch/zhang.chi9/test/'
train_generator = traingen.flow_from_directory(train)
test_generator = testgen.flow_from_directory(test)

In [ ]:
# Crate a classifer with transfer learning from Xception network. 
base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    include_top=False) 

# Freeze the base layer and only train the top layers
base_model.trainable = False

inputs = keras.Input(shape=(None, None, 3))
x = base_model(inputs)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.8)(x)
outputs = keras.layers.Dense(25,activation='softmax')(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer=keras.optimizers.Adam(),
              loss=categorical_crossentropy,
              metrics=[categorical_accuracy])

# Start training the neural nets and save the model if the validation accuracy improve
weights_path = '/scratch/zhang.chi9/arch_weights/'
best_acc = ModelCheckpoint(f'{weights_path}best_acc_model.h5', monitor='val_categorical_accuracy',save_best_only=True, mode='max',verbose = 1)
model.fit_generator(train_generator, validation_data = test_generator, epochs = 10,callbacks=[best_acc])

In [ ]:
# Unfreeze the base layer, use a very small learning rate to fine tuning the base layers
base_model.trainable = True
model.compile(optimizer=keras.optimizers.Adam(1e-5),
              loss=categorical_crossentropy,
              metrics=[categorical_accuracy])
weights_path = '/scratch/zhang.chi9/arch_weights/'
best_acc = ModelCheckpoint(f'{weights_path}best_acc_model.h5', monitor='val_categorical_accuracy',save_best_only=True, mode='max',verbose = 1)

model.fit_generator(train_generator, validation_data = test_generator, epochs = 20,callbacks=[best_acc])